## Sites_dim
Code to generate sites.csv as input to the WaDE db for WA water rights

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from dateutil.parser import parse
from pyproj import CRS, Transformer, Proj
from utilityFunctions import *

In [2]:
# working directory
working_dir = "ProcessedInputData"
os.chdir(working_dir)

In [3]:
# Input files
fileInput1 = "D_Point.csv"

# output sites
out_sitdim = 'sites.csv'

In [4]:
#column names
#10.24.19 rename 'WaDESiteUUID' to 'SiteUUID'
columns=['WaDESiteUUID', 'SiteNativeID', 'SiteName', 'USGSSiteID', 'SiteTypeCV', 'Longitude', 'Latitude',
          'SitePoint', 'SiteNativeURL', 'Geometry', 'CoordinateMethodCV', 'CoordinateAccuracy', 'GNISCodeCV',
          'EPSGCodeCV', 'NHDNetworkStatusCV', 'NHDProductCV', 'NHDUpdateDate', 'NHDReachCode', 'NHDMeasureNumber',
          'StateCV']

# These are not used currently. Data types inferred from the inputs
dtypesx = ['NVarChar(55)	NVarChar(50)	NVarChar(500)	NVarChar(250)	NVarChar(100)	Double	Double	Geometry',
           'NVarChar(250)	Geometry	NVarChar(100)	NVarChar(255)	NVarChar(50)	NVarChar(50)	NVarChar(50)',
           'NVarChar(50)	Date	NVarChar(50)	NVarChar(50)	NChar(5)']

In [5]:
# create target dataframe

#assumes dtypes inferred from CO file
outdf100=pd.DataFrame(columns=columns)

In [6]:
print("Reading inputs...")

# Read Inputs and merge tables

# points of diversion 
df100 = pd.read_csv(fileInput1,encoding = "ISO-8859-1") #, or alternatively encoding = "utf-8"
print (len(df100.index))

#df100 = df100.head(10000) #only runs first 100 lines for testing.

df100.head(5)

Reading inputs...
155062


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (9,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,X,Y,OBJECTID,D_Point_ID,D_Point_Type_CD,Location_CD,Assoc_FL,Misc_CD,Position_With_CD,Active_DT,Inactive_DT,Update_TD,Update_User_ID,Comment_DS,Created_TD,Created_User_ID
0,1665873.059,454923.2363,1,200801.0,WL,U,N,,S,NaN,NaN,2013-03-28T09:58:00.000,"""ECY\DKRO461""",NaN,NaN,NaN
1,1816741.203,456744.0551,2,200889.0,MW,G,Y,,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1813847.794,456733.0063,3,200890.0,MW,G,Y,,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1571475.128,457350.1307,4,201092.0,WL,U,Y,,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1715391.547,402037.7843,5,201191.0,MW,G,Y,,S,NaN,NaN,2010-06-11T11:28:19.000,"""ECY\DKRO461""",NaN,NaN,NaN


In [7]:
list(df100.columns) 

['X',
 'Y',
 'OBJECTID',
 'D_Point_ID',
 'D_Point_Type_CD',
 'Location_CD',
 'Assoc_FL',
 'Misc_CD',
 'Position_With_CD',
 'Active_DT',
 'Inactive_DT',
 'Update_TD',
 'Update_User_ID',
 'Comment_DS',
 'Created_TD',
 'Created_User_ID']

In [8]:
print("SiteTypeCV Dictionary and function...")

SiteTypeCVDictWA = {
    "GC":"ground water collector",
    "HW":"headworks gravity flow (or surface water device unknown)",
    "ID":"irrigation dam",
    "MW":"monitoring well",
    "PM":"surface water pump",
    "RD":"reservoir dam",
    "WL":"well (or ground water device unknown)"
}

# Get SiteTypeCV based on the field "D_Point_Type_CD" and map Blank to “Unknown”
def assignSiteTypeCVWA(colrowValue):
    # may need to modify capitalization in beneficialUseDictionary
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = 'Unknown'
    else:
        keyStr = colrowValue.strip()  # remove whitespace chars
        try:
            outList = SiteTypeCVDictWA[keyStr]
        except:
            outList = 'Unknown'

    return outList

SiteTypeCV Dictionary and function...


In [9]:
print("SiteTypeCV...")

df100 = df100.assign(SiteTypeCV='')  #add new column and make it empty

# no-loop approach?
df100['SiteTypeCV'] = df100.apply(lambda row: assignSiteTypeCVWA(row['D_Point_Type_CD']), axis=1)

SiteTypeCV...


In [10]:
print("Project to longitude/ latitude  ")

df100 = df100.assign(Longitude='')
df100 = df100.assign(Latitude='')

# use pyproj to project to lat lon
crs_to = CRS('EPSG:4326')  # CRS("WGS84")
# Projection: NAD83 HARN / Washington South (ftUS)
crs_from = CRS("EPSG:2927")  
transformer = Transformer.from_crs(crs_from, crs_to)

# drop cells with no x or y coordinate
df100 = df100.replace(np.nan, '') 
dropIndex = df100.loc[(df100['X'] == '') | (df100['Y'] == '')].index
if len(dropIndex) > 0:
    df100 = df100.drop(dropIndex)
    df100 = df100.reset_index(drop=True)

lonList = []
latList = []
for ix in range(len(df100.index)):
    #print(ix)
    x1 = df100.loc[ix, 'X']
    y1 = df100.loc[ix, 'Y']
    try:
        lat, lon  = transformer.transform(float(x1), float(y1))
        lonList.append(lon)
        latList.append(lat)
    except:
        lonList.append('')
        latList.append('')

df100['Longitude'] = lonList
df100['Latitude'] = latList

df100

Project to longitude/ latitude  


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,X,Y,OBJECTID,D_Point_ID,D_Point_Type_CD,Location_CD,Assoc_FL,Misc_CD,Position_With_CD,Active_DT,Inactive_DT,Update_TD,Update_User_ID,Comment_DS,Created_TD,Created_User_ID,SiteTypeCV,Longitude,Latitude
0,1665873.059,454923.2363,1,200801,WL,U,N,,S,,,2013-03-28T09:58:00.000,"""ECY\DKRO461""",,,,well (or ground water device unknown),-120.398762,46.580802
1,1816741.203,456744.0551,2,200889,MW,G,Y,,S,,,,,,,,monitoring well,-119.798724,46.583691
2,1813847.794,456733.0063,3,200890,MW,G,Y,,S,,,,,,,,monitoring well,-119.810231,46.583730
3,1571475.128,457350.1307,4,201092,WL,U,Y,,S,,,,,,,,well (or ground water device unknown),-120.774208,46.587173
4,1715391.547,402037.7843,5,201191,MW,G,Y,,S,,,2010-06-11T11:28:19.000,"""ECY\DKRO461""",,,,monitoring well,-120.202625,46.435437
5,1775438.262,387648.1805,6,201247,WL,P,Y,,S,,,,,,,,well (or ground water device unknown),-119.964852,46.395109
6,1801913.726,426459.3614,7,201322,MW,G,Y,,S,,,,,,,,monitoring well,-119.858672,46.500993
7,1651078.242,433514.4569,8,201408,MW,G,Y,,S,,,,,,,,monitoring well,-120.457645,46.522133
8,1671546.021,434691.1950,9,201412,MW,G,Y,,S,,,,,,,,monitoring well,-120.376327,46.525300
9,1671997.977,433022.5353,10,201415,WL,U,Y,,S,,,,,,,,well (or ground water device unknown),-120.374542,46.520723


In [15]:
print("CoordinateMethodCV dictionary and function...")

coordMethodCVDictWA = {
    "BLMPLS":"Bureau of Land Management's Public Land Survey System",
    "DC_SEC":"Douglas County Sections",
    "M":"paper map or manuscript",
    "N":"not applicable (ex., when fieldchecked withGPS)",
    "O":"orthophotos (DOQQ black and white)",
    "Q":"quads (DRG24 topos)",
    "S":"sections (DNR)",
    "T":"plstiger"
}

#coordMethodCVDictWA_var = {
'''
    "P########":"parcels by year month day or by year (ex., P20040409 or P2006)"
    "CYO####":"County Yakima Orthos by year (ex., CYO2011)",
    "ISSW####":"Image Services Statewide (ex., ISSW2011)",
    "NAIP####":"NAIP photos by year (ex., NAIP2006)"
'''
#}

# Get coord method based on the field "Position_With_CD" 
def assigncoordMethodCVWA(colrowValue):
    # may need to modify capitalization in beneficialUseDictionary
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        keyStr = colrowValue.strip()  # remove whitespace chars
        try:
            if keyStr[0] == 'P' and len(keyStr) == 9:
                    outList = "parcels by year month day " + keyStr[1:9]
            elif keyStr[0] == 'P' and len(keyStr) == 5:
                    outList = "parcels by year " + keyStr[1:5]
            elif keyStr[0:3] == 'CYO':
                outList = "County Yakima Orthos by year " + keyStr[3:7]
            elif keyStr[0:4] == 'ISSW':
                outList = "Image Services Statewide " + keyStr[4:8]
            elif keyStr[0:4] == 'NAIP':
                outList = "NAIP photos by year " + keyStr[4:8]
            else:
                outList = coordMethodCVDictWA[keyStr]
        except:
            outList = ''

    return outList

CoordinateMethodCV dictionary and function...


In [17]:
print("CoordinateMethodCV...")

df100 = df100.assign(CoordinateMethodCV='')  #add new column and make it empty

# no-loop approach?
df100['CoordinateMethodCV'] = df100.apply(lambda row: assigncoordMethodCVWA(row['Position_With_CD']), 
                                                      axis=1)

df100

CoordinateMethodCV...


,X,Y,OBJECTID,D_Point_ID,D_Point_Type_CD,Location_CD,Assoc_FL,Misc_CD,Position_With_CD,Active_DT,Inactive_DT,Update_TD,Update_User_ID,Comment_DS,Created_TD,Created_User_ID,SiteTypeCV,Longitude,Latitude,CoordinateMethodCV
0,1665873.059,454923.2363,1,200801,WL,U,N,,S,,,2013-03-28T09:58:00.000,"""ECY\DKRO461""",,,,well (or ground water device unknown),-120.398762,46.580802,sections (DNR)
1,1816741.203,456744.0551,2,200889,MW,G,Y,,S,,,,,,,,monitoring well,-119.798724,46.583691,sections (DNR)
2,1813847.794,456733.0063,3,200890,MW,G,Y,,S,,,,,,,,monitoring well,-119.810231,46.583730,sections (DNR)
3,1571475.128,457350.1307,4,201092,WL,U,Y,,S,,,,,,,,well (or ground water device unknown),-120.774208,46.587173,sections (DNR)
4,1715391.547,402037.7843,5,201191,MW,G,Y,,S,,,2010-06-11T11:28:19.000,"""ECY\DKRO461""",,,,monitoring well,-120.202625,46.435437,sections (DNR)
5,1775438.262,387648.1805,6,201247,WL,P,Y,,S,,,,,,,,well (or ground water device unknown),-119.964852,46.395109,sections (DNR)
6,1801913.726,426459.3614,7,201322,MW,G,Y,,S,,,,,,,,monitoring well,-119.858672,46.500993,sections (DNR)
7,1651078.242,433514.4569,8,201408,MW,G,Y,,S,,,,,,,,monitoring well,-120.457645,46.522133,sections (DNR)
8,1671546.021,434691.1950,9,201412,MW,G,Y,,S,,,,,,,,monitoring well,-120.376327,46.525300,sections (DNR)
9,1671997.977,433022.5353,10,201415,WL,U,Y,,S,,,,,,,,well (or ground water device unknown),-120.374542,46.520723,sections (DNR)


In [18]:
print("Coordinate Accuracy dictionary and function...")

coordinateAccuracyDictWA = {
    "C":"field checked (without GPS)",
    "G":"field checked with GPS",
    "P":"proposed (does not exist in real world)",
    "PA":"proposed and All-right (does not exist in real world)",
    "PD":"proposed and Dubious (does not exist in real world)",
    "PM":"proposed and Multiple Dubious (does not exist in real world)",
    "PX":"proposed and Centroid Dubious (does not exist in real world)",
    "U":"unchecked",
    "UA":"unchecked and All-right",
    "UD":"unchecked and Dubious",
    "UM":"unchecked and Multiple Dubious",
    "UX":"unchecked and Centroid Dubious",
    "W":"from well log, unchecked",
    "WA":"from well log, unchecked and All-right",
    "WD":"from well log, unchecked and Dubious",
    "WX":"from well log, unchecked and Centroid Dubious"
}

# Get coord accuracy based on the field "Location_CD" 
def assigncoordAccuracyWA(colrowValue):
    # may need to modify capitalization in beneficialUseDictionary
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        keyStr = colrowValue.strip()  # remove whitespace chars
        try:
            outList = coordinateAccuracyDictWA[keyStr]
        except:
            outList = ''

    return outList

Coordinate Accuracy dictionary and function...


In [21]:
print("Coordinate Accuracy...")

df100 = df100.assign(CoordinateAccuracy='')  #add new column and make it empty

# no-loop approach?
df100['CoordinateAccuracy'] = df100.apply(lambda row: assigncoordAccuracyWA(row['Location_CD']),
                                                      axis=1)

df100

Coordinate Accuracy...


,X,Y,OBJECTID,D_Point_ID,D_Point_Type_CD,Location_CD,Assoc_FL,Misc_CD,Position_With_CD,Active_DT,...,Update_TD,Update_User_ID,Comment_DS,Created_TD,Created_User_ID,SiteTypeCV,Longitude,Latitude,CoordinateMethodCV,CoordinateAccuracy
0,1665873.059,454923.2363,1,200801,WL,U,N,,S,,...,2013-03-28T09:58:00.000,"""ECY\DKRO461""",,,,well (or ground water device unknown),-120.398762,46.580802,sections (DNR),unchecked
1,1816741.203,456744.0551,2,200889,MW,G,Y,,S,,...,,,,,,monitoring well,-119.798724,46.583691,sections (DNR),field checked with GPS
2,1813847.794,456733.0063,3,200890,MW,G,Y,,S,,...,,,,,,monitoring well,-119.810231,46.583730,sections (DNR),field checked with GPS
3,1571475.128,457350.1307,4,201092,WL,U,Y,,S,,...,,,,,,well (or ground water device unknown),-120.774208,46.587173,sections (DNR),unchecked
4,1715391.547,402037.7843,5,201191,MW,G,Y,,S,,...,2010-06-11T11:28:19.000,"""ECY\DKRO461""",,,,monitoring well,-120.202625,46.435437,sections (DNR),field checked with GPS
5,1775438.262,387648.1805,6,201247,WL,P,Y,,S,,...,,,,,,well (or ground water device unknown),-119.964852,46.395109,sections (DNR),proposed (does not exist in real world)
6,1801913.726,426459.3614,7,201322,MW,G,Y,,S,,...,,,,,,monitoring well,-119.858672,46.500993,sections (DNR),field checked with GPS
7,1651078.242,433514.4569,8,201408,MW,G,Y,,S,,...,,,,,,monitoring well,-120.457645,46.522133,sections (DNR),field checked with GPS
8,1671546.021,434691.1950,9,201412,MW,G,Y,,S,,...,,,,,,monitoring well,-120.376327,46.525300,sections (DNR),field checked with GPS
9,1671997.977,433022.5353,10,201415,WL,U,Y,,S,,...,,,,,,well (or ground water device unknown),-120.374542,46.520723,sections (DNR),unchecked


In [23]:
print("Direct mapping columns...")
#
# directly mapped cells
# 
destCols=['SiteNativeID', 'SiteTypeCV', 'Longitude', 'Latitude', 'CoordinateMethodCV',
          'CoordinateAccuracy']
srsCols=['D_Point_ID', 'SiteTypeCV', 'Longitude', 'Latitude', 'CoordinateMethodCV',
         'CoordinateAccuracy']

outdf100[destCols] = df100[srsCols]

# replace NaN with blank cells
outdf100 = outdf100.replace(np.nan, '') 

outdf100

Direct mapping columns...


,WaDESiteUUID,SiteNativeID,SiteName,USGSSiteID,SiteTypeCV,Longitude,Latitude,SitePoint,SiteNativeURL,Geometry,CoordinateMethodCV,CoordinateAccuracy,GNISCodeCV,EPSGCodeCV,NHDNetworkStatusCV,NHDProductCV,NHDUpdateDate,NHDReachCode,NHDMeasureNumber,StateCV
0,,200801,,,well (or ground water device unknown),-120.398762,46.580802,,,,sections (DNR),unchecked,,,,,,,,
1,,200889,,,monitoring well,-119.798724,46.583691,,,,sections (DNR),field checked with GPS,,,,,,,,
2,,200890,,,monitoring well,-119.810231,46.583730,,,,sections (DNR),field checked with GPS,,,,,,,,
3,,201092,,,well (or ground water device unknown),-120.774208,46.587173,,,,sections (DNR),unchecked,,,,,,,,
4,,201191,,,monitoring well,-120.202625,46.435437,,,,sections (DNR),field checked with GPS,,,,,,,,
5,,201247,,,well (or ground water device unknown),-119.964852,46.395109,,,,sections (DNR),proposed (does not exist in real world),,,,,,,,
6,,201322,,,monitoring well,-119.858672,46.500993,,,,sections (DNR),field checked with GPS,,,,,,,,
7,,201408,,,monitoring well,-120.457645,46.522133,,,,sections (DNR),field checked with GPS,,,,,,,,
8,,201412,,,monitoring well,-120.376327,46.525300,,,,sections (DNR),field checked with GPS,,,,,,,,
9,,201415,,,well (or ground water device unknown),-120.374542,46.520723,,,,sections (DNR),unchecked,,,,,,,,


In [26]:
print("Empty lat/lon")
#TODO there are too many empty location coordinates so we are not dropping them here

outdf100purge = outdf100.loc[(outdf100['Longitude'] == '') | (outdf100['Longitude'] == np.nan)
                             | (outdf100['Latitude'] == '') | (outdf100['Latitude'] == np.nan)]
if len(outdf100purge.index) > 0:
    outdf100purge.to_csv('sites_latlon_missing.csv')    #index=False,
    dropIndex = outdf100purge.index
    outdf100 = outdf100.drop(dropIndex) 
    outdf100 = outdf100.reset_index(drop=True)
    
outdf100

Empty lat/lon


,WaDESiteUUID,SiteNativeID,SiteName,USGSSiteID,SiteTypeCV,Longitude,Latitude,SitePoint,SiteNativeURL,Geometry,CoordinateMethodCV,CoordinateAccuracy,GNISCodeCV,EPSGCodeCV,NHDNetworkStatusCV,NHDProductCV,NHDUpdateDate,NHDReachCode,NHDMeasureNumber,StateCV
0,,200801,,,well (or ground water device unknown),-120.398762,46.580802,,,,sections (DNR),unchecked,,,,,,,,
1,,200889,,,monitoring well,-119.798724,46.583691,,,,sections (DNR),field checked with GPS,,,,,,,,
2,,200890,,,monitoring well,-119.810231,46.583730,,,,sections (DNR),field checked with GPS,,,,,,,,
3,,201092,,,well (or ground water device unknown),-120.774208,46.587173,,,,sections (DNR),unchecked,,,,,,,,
4,,201191,,,monitoring well,-120.202625,46.435437,,,,sections (DNR),field checked with GPS,,,,,,,,
5,,201247,,,well (or ground water device unknown),-119.964852,46.395109,,,,sections (DNR),proposed (does not exist in real world),,,,,,,,
6,,201322,,,monitoring well,-119.858672,46.500993,,,,sections (DNR),field checked with GPS,,,,,,,,
7,,201408,,,monitoring well,-120.457645,46.522133,,,,sections (DNR),field checked with GPS,,,,,,,,
8,,201412,,,monitoring well,-120.376327,46.525300,,,,sections (DNR),field checked with GPS,,,,,,,,
9,,201415,,,well (or ground water device unknown),-120.374542,46.520723,,,,sections (DNR),unchecked,,,,,,,,


In [27]:
print("Dropping duplicates...")
#filter the whole table based on a unique combination of site ID, SiteName, SiteType
#10.24.19 added lat lon to list
print(len(outdf100.index))
outdf100 = outdf100.drop_duplicates(subset=['SiteNativeID', 'SiteName', 'SiteTypeCV', 'Longitude', 'Latitude'])   #
outdf100 = outdf100.reset_index(drop=True)
print(len(outdf100.index))

Dropping duplicates...
155062
155059


In [28]:
# hardcoded columns
print("Hard coded")

outdf100.EPSGCodeCV = 'EPSG:4326'
outdf100.SiteName = "Unspecified"


Hard coded


In [30]:
print("Check Site Native IDs are duplicated")

siteNIdDup = False
siteNativeIDdup=outdf100.loc[outdf100.duplicated(subset=['SiteNativeID'])]
if len(siteNativeIDdup.index) > 0:
    print("Site Native IDs are duplicated")
    siteNIdDup = True
#outdf100

print(len(siteNativeIDdup))

Check Site Native IDs are duplicated
Site Native IDs are duplicated
20


In [31]:
print("Adding SiteUUID...")

if siteNIdDup:    
    # 10.24.19 create unique site uuid
    outdf100 = outdf100.reset_index(drop=True)
    outdf100['TempUUID'] = range(1, len(outdf100.index) + 1)
    #append 'NM'
    outdf100['WaDESiteUUID'] = outdf100.apply(lambda row: "_".join(["WA", str(row['TempUUID'])]),
                                          axis=1)
    #drop temp uuid
    outdf100 = outdf100.drop('TempUUID', axis=1)
else:
    #append 'NM'
    outdf100['WaDESiteUUID'] = outdf100.apply(lambda row: '' if str(row['SiteNativeID']) == '' 
                                        else "_".join(["WA", str(row['SiteNativeID'])]), axis=1)

outdf100

Adding SiteUUID...


,WaDESiteUUID,SiteNativeID,SiteName,USGSSiteID,SiteTypeCV,Longitude,Latitude,SitePoint,SiteNativeURL,Geometry,CoordinateMethodCV,CoordinateAccuracy,GNISCodeCV,EPSGCodeCV,NHDNetworkStatusCV,NHDProductCV,NHDUpdateDate,NHDReachCode,NHDMeasureNumber,StateCV
0,WA_1,200801,Unspecified,,well (or ground water device unknown),-120.398762,46.580802,,,,sections (DNR),unchecked,,EPSG:4326,,,,,,
1,WA_2,200889,Unspecified,,monitoring well,-119.798724,46.583691,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
2,WA_3,200890,Unspecified,,monitoring well,-119.810231,46.583730,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
3,WA_4,201092,Unspecified,,well (or ground water device unknown),-120.774208,46.587173,,,,sections (DNR),unchecked,,EPSG:4326,,,,,,
4,WA_5,201191,Unspecified,,monitoring well,-120.202625,46.435437,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
5,WA_6,201247,Unspecified,,well (or ground water device unknown),-119.964852,46.395109,,,,sections (DNR),proposed (does not exist in real world),,EPSG:4326,,,,,,
6,WA_7,201322,Unspecified,,monitoring well,-119.858672,46.500993,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
7,WA_8,201408,Unspecified,,monitoring well,-120.457645,46.522133,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
8,WA_9,201412,Unspecified,,monitoring well,-120.376327,46.525300,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
9,WA_10,201415,Unspecified,,well (or ground water device unknown),-120.374542,46.520723,,,,sections (DNR),unchecked,,EPSG:4326,,,,,,


In [32]:
print("Droping duplicates...")

# replace NaN with blank cells
outdf100 = outdf100.replace(np.nan, '')
#drop duplicate rows; just make sure
outdf100Duplicated=outdf100.loc[outdf100.duplicated()]
if len(outdf100Duplicated.index) > 0:
    outdf100Duplicated.to_csv("sites_duplicaterows.csv")  # index=False,
    outdf100.drop_duplicates(inplace=True)   #
    outdf100 = outdf100.reset_index(drop=True)

outdf100

Droping duplicates...


,WaDESiteUUID,SiteNativeID,SiteName,USGSSiteID,SiteTypeCV,Longitude,Latitude,SitePoint,SiteNativeURL,Geometry,CoordinateMethodCV,CoordinateAccuracy,GNISCodeCV,EPSGCodeCV,NHDNetworkStatusCV,NHDProductCV,NHDUpdateDate,NHDReachCode,NHDMeasureNumber,StateCV
0,WA_1,200801,Unspecified,,well (or ground water device unknown),-120.398762,46.580802,,,,sections (DNR),unchecked,,EPSG:4326,,,,,,
1,WA_2,200889,Unspecified,,monitoring well,-119.798724,46.583691,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
2,WA_3,200890,Unspecified,,monitoring well,-119.810231,46.583730,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
3,WA_4,201092,Unspecified,,well (or ground water device unknown),-120.774208,46.587173,,,,sections (DNR),unchecked,,EPSG:4326,,,,,,
4,WA_5,201191,Unspecified,,monitoring well,-120.202625,46.435437,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
5,WA_6,201247,Unspecified,,well (or ground water device unknown),-119.964852,46.395109,,,,sections (DNR),proposed (does not exist in real world),,EPSG:4326,,,,,,
6,WA_7,201322,Unspecified,,monitoring well,-119.858672,46.500993,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
7,WA_8,201408,Unspecified,,monitoring well,-120.457645,46.522133,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
8,WA_9,201412,Unspecified,,monitoring well,-120.376327,46.525300,,,,sections (DNR),field checked with GPS,,EPSG:4326,,,,,,
9,WA_10,201415,Unspecified,,well (or ground water device unknown),-120.374542,46.520723,,,,sections (DNR),unchecked,,EPSG:4326,,,,,,


In [33]:
print("Checking required isnot null...")
# check if any cell of these columns is null
requiredCols = ['WaDESiteUUID', 'SiteName', 'CoordinateMethodCV', 'GNISCodeCV', 'EPSGCodeCV']

# replace NaN with blank cells
outdf100 = outdf100.replace(np.nan, '')

outdf100_nullMand = outdf100.loc[(outdf100["WaDESiteUUID"] == '') |
                                 (outdf100["SiteName"] == '') | 
                                 (outdf100["CoordinateMethodCV"] == '') |
                                 (outdf100["GNISCodeCV"] == '') | 
                                 (outdf100["EPSGCodeCV"] == '')]

if (len(outdf100_nullMand.index) > 0):
    outdf100_nullMand.to_csv('sites_mandatoryFieldMissing.csv')  # index=False,

# ToDO: purge these cells if there is any missing? #For now left to be inspected and reported

Checking required isnot null...


In [34]:
print("Writing out...")

#write out
outdf100.to_csv(out_sitdim, index=False, encoding = "utf-8")

print("Done sites")

Writing out...
Done sites
